In [1]:
import os

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import pickle
import numpy as np
import pandas as pd

import cv2
import skimage.io


from skimage.color import label2rgb, gray2rgb, rgb2gray, rgb2hsv

from skimage.transform import resize, rotate, rescale

from skimage.util import img_as_ubyte, crop

from skimage.filters import (laplace, prewitt, sobel, roberts, median,
gaussian, threshold_otsu, threshold_multiotsu,  difference_of_gaussians,
threshold_isodata, threshold_mean, threshold_yen, threshold_sauvola,
threshold_niblack, threshold_triangle, threshold_li, threshold_local)

from skimage.segmentation import clear_border

from skimage.measure import label, regionprops, regionprops_table, find_contours

from skimage.morphology import (disk, square, closing, binary_opening,
                        binary_closing, opening, binary_dilation, binary_erosion, remove_small_objects)

# Load data

In [3]:
train_data_path = os.path.join('data', 'train_games')
train_game_count = 7
game_round_count = 13

dict_data = {}

for i in range(1, train_game_count+1):
    # For each game, store the path of the csv and the image of each round
    dict_data[f'game{i}'] = {}
    dict_data[f'game{i}']['url'] = train_data_path + f'/game{i}'
    dict_data[f'game{i}']['csv'] = train_data_path + f'/game{i}.csv'
    
    for j in range(1, game_round_count+1):
        dict_data[f'game{i}'][f'round{j}'] = skimage.io.imread(train_data_path + f'/game{i}/{j}.jpg')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,8))
ax.imshow(dict_data['game3']['round7'])
ax.set_title('Game1 Round1')
ax.axis('off')
plt.show()

# Segmentation function

In [ ]:
#Input: coloured image of the round
#Output: DataFrame containing all segmented objects (cards + dealer + random objects)
#with geometrical properties
def find_potential_objects_in_original_round_image(coloured_image):
    hsv_img = rgb2hsv(coloured_image) 
    hsv_img = img_as_ubyte(hsv_img) # Convert 0-1 to 0-255 
    
    thres_brightness = threshold_multiotsu(hsv_img[:,:,2]).max() # threshold for the "value" channel
    
    # HSV image is filtered on the 3 channels:
    # H: [145-200] white colour , S: [0-255] , V: [threshold-255] high values
    filtered_im = cv2.inRange(hsv_img, np.array([145,0,thres_brightness]), np.array([200,255,255])) # Binary filtered image
    
    labeled_im = label(filtered_im, background=None, connectivity=filtered_im.ndim) # Find objects using region growing and labeling
    labeled_im = remove_small_objects(labeled_im, min_size=30000) # Remove objects smaller than the dealer and the cards
    labeled_im = median(labeled_im, square(15)) # Remove noise
    
    # Dictionnary that stores information for each segmented objects 
    props_objs = regionprops_table(labeled_im, properties=(
                                                  "label",
                                                  "area",
                                                  "filled_area",
                                                  "major_axis_length",
                                                  "minor_axis_length",
                                                  'centroid',
                                                  "slice",
                                                  "image"
                                                   ))
    df_objs = pd.DataFrame(props_objs)
    df_objs["labeled_im"] = 0 # random value
    df_objs["labeled_im"] = df_objs["labeled_im"].apply(lambda x: labeled_im)
    
    
    return df_objs

In [ ]:
#Input: DataFrame containing all the potential objects 
#Output: DataFrame with 4 objects corresponding to the 4 cards (unordered)

def select_cards_from_potential_objects(df_objs): 
    
    # Features differentiating cards from the rest of the objects
    # These are reference values obtained by exploring the dataset
    C_REF_MAJOR_AXIS_MEAN = 838
    C_REF_MAJOR_AXIS_STD = 16
    C_REF_MINOR_AXIS_MEAN = 542
    C_REF_MINOR_AXIS_STD = 23
    
    cards_feature1_series = (df_objs["major_axis_length"]-C_REF_MAJOR_AXIS_MEAN)/C_REF_MAJOR_AXIS_STD # Normalized feature 1 
    cards_feature2_series = (df_objs["minor_axis_length"]-C_REF_MINOR_AXIS_MEAN)/C_REF_MINOR_AXIS_STD # Normalized feature 2
    
    # Euclidian distance corresponding to the similarity between an object and the reference card.
    # The smaller the distance, the greater the similarity. Here the 4 most similar objects are kept.
    cards_similarity_distances_series = (cards_feature1_series**2+cards_feature2_series**2)**0.5 # Euclidian distance
    df_objs["card_similarity_measure"] = cards_similarity_distances_series
    
    return df_objs.nsmallest(4, "card_similarity_measure")